In [1]:
!nvidia-smi

Fri Jul  4 15:28:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P8             12W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
from diffusers import StableDiffusionXLPipeline, StableDiffusionPipeline
import torch
from PIL import Image
import gradio as gr 

In [3]:
!pip install --upgrade gradio

In [12]:
AVAILABLE_MODELS = {
    "Stable Diffusion v1.5": "sd-legacy/stable-diffusion-v1-5",
    "SDXL ": "stabilityai/stable-diffusion-xl-base-1.0"}

loaded_pipelines = {}



for name, model_id in AVAILABLE_MODELS.items():
    if name == "Stable Diffusion v1.5":
        current_pipe = StableDiffusionPipeline.from_pretrained("sd-legacy/stable-diffusion-v1-5", torch_dtype=torch.float16)
        current_pipe.to("cuda")
        loaded_pipelines[name] = current_pipe
    else:
        current_pipe = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, added_cond_kwargs={})
        current_pipe.to("cuda")
        loaded_pipelines[name] = current_pipe




Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Keyword arguments {'added_cond_kwargs': {}} are not expected by StableDiffusionXLPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
def generate_image(prompt,selected_model_name, cfg_scale, output_width, output_height):
    if selected_model_name not in loaded_pipelines:
        raise gr.Error(f"Selected model '{selected_model_name}' not loaded. Please try again or select another.")
    
    current_pipe = loaded_pipelines[selected_model_name]
    image = current_pipe(
        prompt,
        width=output_width,    # Pass the width argument
        height=output_height , # Use the resized input image
        guidance_scale=cfg_scale
    ).images[0]
    return image

In [14]:
inputs = [
    gr.Text(label="Prompt", lines=2),
    gr.Dropdown(
    label="Select Stable Diffusion Model",
    choices=list(AVAILABLE_MODELS.keys()), 
    value=list(AVAILABLE_MODELS.keys())[0], 
    interactive=True 
    ),
    gr.Slider(minimum=1.0, maximum=20.0, value=7.5, step=0.5, 
             label="CFG Scale (Guidance Scale)", 
             info="How strongly the prompt influences the generation (Typically 7-12)"),
    gr.Slider(minimum=256, maximum=1024, value=512, step=64, 
             label="Output Width (pixels)", 
             info="Adjust the width of the generated image. (Multiples of 64 recommended)"),
    gr.Slider(minimum=256, maximum=1024, value=512, step=64, 
             label="Output Height (pixels)", 
             info="Adjust the height of the generated image. (Multiples of 64 recommended)")
]


outputs = gr.Image(label="Generated Image", type="pil")

In [16]:
demo = gr.Interface(
    fn=generate_image, 
    inputs=inputs,                 
    outputs=outputs,              
    title="Stable Diffusion Text-to-Image Generator",
    description="Enter a positive prompt for what you want to generate. Adjust parameters and image dimensions."
)

# Launch the interface
demo.launch(debug=True, share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2c2ded5282de715c83.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2c2ded5282de715c83.gradio.live
